In [1]:
import pandas as pd
import numpy as np
from numba import njit

In [6]:
# Data scraped from TV using tvdatafeed
data = pd.read_csv("msft.csv")
data["datetime"] = pd.to_datetime(data["datetime"])
data.set_index("datetime", inplace=True)
data

,symbol,open,high,low,close,volume
datetime,,,,,,
2017-12-19 14:30:00,NASDAQ:MSFT,86.35,86.3500,85.27,85.83,23524787.0
2017-12-20 14:30:00,NASDAQ:MSFT,86.20,86.3000,84.71,85.52,23674931.0
2017-12-21 14:30:00,NASDAQ:MSFT,86.05,86.1000,85.40,85.50,17990745.0
2017-12-22 14:30:00,NASDAQ:MSFT,85.40,85.6300,84.92,85.51,14145841.0
2017-12-26 14:30:00,NASDAQ:MSFT,85.31,85.5346,85.03,85.40,9891237.0
...,...,...,...,...,...,...
2023-02-13 14:30:00,NASDAQ:MSFT,267.64,274.6000,267.15,271.32,44630921.0
2023-02-14 14:30:00,NASDAQ:MSFT,272.67,274.9700,269.28,272.17,37047924.0
2023-02-15 14:30:00,NASDAQ:MSFT,268.32,270.7300,266.18,269.32,28962163.0


In [11]:

@njit
def pine_ema(src, length):
    
    ind = np.empty(shape=src.shape, dtype=float)
    alpha = 2 / (length + 1)
    
    for i in range(len(src)):
        
        if i == 0 or np.isnan(src[i]) or (i > 0 and np.isnan(ind[i-1])):
            ind[i] = src[i]
            continue
            
        ind[i] = (alpha * src[i]) + (1 - alpha) * ind[i-1]
        
    ind[:length -1] = np.nan
    
    return ind
    

In [22]:

pine_ema(pine_ema(data.open.to_numpy(), 15), 15)

array([         nan,          nan,          nan, ..., 250.02299601,
       251.33535166, 252.46597784])

In [14]:
data["ema"] = pine_ema(data.open.to_numpy(), 15)

In [16]:
x = data.open.copy()
x

datetime
2017-12-19 14:30:00     86.35
2017-12-20 14:30:00     86.20
2017-12-21 14:30:00     86.05
2017-12-22 14:30:00     85.40
2017-12-26 14:30:00     85.31
                        ...  
2023-02-13 14:30:00    267.64
2023-02-14 14:30:00    272.67
2023-02-15 14:30:00    268.32
2023-02-16 14:30:00    264.02
2023-02-17 14:30:00    259.39
Name: open, Length: 1300, dtype: float64

In [17]:
x.iloc[:] = pine_ema(data.open.to_numpy(), 15)
x

datetime
2017-12-19 14:30:00           NaN
2017-12-20 14:30:00           NaN
2017-12-21 14:30:00           NaN
2017-12-22 14:30:00           NaN
2017-12-26 14:30:00           NaN
                          ...    
2023-02-13 14:30:00    256.860504
2023-02-14 14:30:00    258.836691
2023-02-15 14:30:00    260.022104
2023-02-16 14:30:00    260.521841
2023-02-17 14:30:00    260.380361
Name: open, Length: 1300, dtype: float64

In [21]:
%%timeit
pd.Series(pine_ema(data.open.to_numpy(), 15), index=data.index)

48.2 µs ± 557 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [19]:
import pandas_ta as ta

In [20]:
%%timeit
ta.ema(data.open, 15)

459 µs ± 32.4 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
